In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/03/07 15:20:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
 print(spark.sparkContext.version)

3.0.3


In [4]:
spark

In [5]:
! wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-03-07 15:20:40--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.105.68
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.105.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv.1’

fhvhv_tripdata_2021 100%[===================>] 699.83M  12.5MB/s    in 60s     

2022-03-07 15:21:40 (11.7 MB/s) - ‘fhvhv_tripdata_2021-02.csv.1’ saved [733822658/733822658]



In [6]:
!wc -l fhvhv_tripdata_2021-02.csv

11613943 fhvhv_tripdata_2021-02.csv


In [8]:
from pyspark.sql import types
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [10]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')\
    .repartition(24)\
    .write.parquet('fhvhv/2021/02/')

In [11]:
!du -h fhvhv

216M	fhvhv/2021/01
210M	fhvhv/2021/02
426M	fhvhv/2021
426M	fhvhv


In [12]:
from pyspark.sql import functions as F
df = spark.read.parquet('fhvhv/2021/02/*')
df.registerTempTable('fhvhv')
spark.sql("""
SELECT
    COUNT(*)
FROM
    fhvhv
WHERE
    DATE(pickup_datetime) = '2021-02-15'
""").show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+



In [15]:
spark.sql("""
SELECT
    pickup_datetime, dropoff_datetime,
    (unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime)) AS duration
FROM
    fhvhv
SORT BY
    duration DESC
""").show()

+-------------------+-------------------+--------+
|    pickup_datetime|   dropoff_datetime|duration|
+-------------------+-------------------+--------+
|2021-02-11 13:40:44|2021-02-12 10:39:44|   75540|
|2021-02-17 15:54:53|2021-02-18 07:48:34|   57221|
|2021-02-25 09:18:18|2021-02-25 18:18:57|   32439|
|2021-02-12 06:16:42|2021-02-12 14:39:10|   30148|
|2021-02-10 15:00:54|2021-02-10 22:49:57|   28143|
|2021-02-09 12:40:43|2021-02-09 20:04:03|   26600|
|2021-02-23 08:02:37|2021-02-23 14:49:56|   24439|
|2021-02-24 10:54:08|2021-02-24 17:28:37|   23669|
|2021-02-05 14:09:56|2021-02-05 20:30:46|   22850|
|2021-02-11 12:36:26|2021-02-11 18:50:42|   22456|
|2021-02-17 10:34:15|2021-02-17 16:39:59|   21944|
|2021-02-26 16:34:50|2021-02-26 22:22:23|   20853|
|2021-02-24 13:20:32|2021-02-24 19:01:48|   20476|
|2021-02-10 15:23:08|2021-02-10 21:01:32|   20304|
|2021-02-08 18:26:37|2021-02-08 23:43:52|   19035|
|2021-02-05 11:02:24|2021-02-05 16:17:28|   18904|
|2021-02-16 17:02:02|2021-02-16

In [17]:
spark.sql("""
SELECT
    dispatching_base_num,
    COUNT(dispatching_base_num) AS frequent
FROM
    fhvhv
GROUP BY
    dispatching_base_num
ORDER BY
    frequent DESC
""").show()

+--------------------+--------+
|dispatching_base_num|frequent|
+--------------------+--------+
|              B02510| 3233664|
|              B02764|  965568|
|              B02872|  882689|
|              B02875|  685390|
|              B02765|  559768|
|              B02869|  429720|
|              B02887|  322331|
|              B02871|  312364|
|              B02864|  311603|
|              B02866|  311089|
|              B02878|  305185|
|              B02682|  303255|
|              B02617|  274510|
|              B02883|  251617|
|              B02884|  244963|
|              B02882|  232173|
|              B02876|  215693|
|              B02879|  210137|
|              B02867|  200530|
|              B02877|  198938|
+--------------------+--------+
only showing top 20 rows

